# 📺 Denoise a map with custom inputs

This notebook takes you step by step through the same procedure as used by `meteor.diffmap`, but in a notebook. There is one difference: we assume you've got some pre-computed phases that you want to employ, rather than computing phases from a structural model (CIF/PDB).

In [20]:
# ruff: noqa: T201

import numpy as np
import reciprocalspaceship as rs
from matplotlib import pyplot as plt
import sys
sys.path.append('/Users/vapostolop/meteor')


In [21]:
from meteor.diffmaps import compute_difference_map, max_negentropy_kweighted_difference_map
from meteor.rsmap import Map
from meteor.tv import tv_denoise_difference_map
from meteor.validate import map_negentropy
from meteor.lpGaussian import low_pass_denoise_difference_map

### 1. Load the data of interest

We'll use `scaled-test-data.mtz`, an test/example MTZ provided in this repo. The data come from a real experiment, Fadini _et al._ (**2023**) _J Am Chem Soc_ 145: 15796-15808 (https://doi.org/10.1021/jacs.3c02313), where the authors imaged the cis-trans isomerization of rsEGFP2. The MTZ contains a lot of columns -- we'll ignore most of them and focus on.

- the amplitudes and phases for a dark reference dataset
- amplitudes for a light-activated dataset

In [22]:
# path is relative to where this notebook usually is in the `meteor` repo
mtz_path = "../test/data/scaled-test-data.mtz"
mtz_dataset = rs.read_mtz(mtz_path)

print(mtz_dataset.columns)

Index(['F_on', 'SIGF_on', 'F_off', 'SIGF_off', 'FC_nochrom', 'PHIC_nochrom',
       'F_on_scaled', 'SIGF_on_scaled', 'F_off_scaled', 'SIGF_off_scaled',
       'F_k', 'SIGF_k', 'PHI_k', 'F_TV', 'PHI_TV', 'SIGF_TV', 'F_itTV',
       'SIGF_itTV', 'PHI_itTV'],
      dtype='object')


We'll use the same phase column for both the `native` and `derivative` datasets we're comparing, making the maps we compute isomorphous difference maps. Note that's not a requirement, though!

In this case, the phases were computed _omitting_ the chromophore where isomerization was expected, so the results should not be biased by the starting model at all. This is a nice application of a custom phase calculation. But: the same effect could be achieved by passing a model without the chromophore to `meteor.diffmap` 😊.

In [23]:
native_map = Map(
    mtz_dataset,
    amplitude_column="F_off_scaled",
    phase_column="PHIC_nochrom",
    uncertainty_column="SIGF_off_scaled",
)

derivative_map = Map(
    mtz_dataset,
    amplitude_column="F_on_scaled",
    phase_column="PHIC_nochrom",
    uncertainty_column="SIGF_on_scaled",
)

In [24]:
# for a reference, let's compute a "vanilla" difference map to benchmark the initial negentropy
vanilla_isomorphous_diffmap = compute_difference_map(derivative_map, native_map)
initial_negentropy = map_negentropy(vanilla_isomorphous_diffmap)

print(f"initial negentropy: {initial_negentropy:.5f}")

initial negentropy: 0.00025


In [33]:
vanilla_isomorphous_diffmap.columns

Index(['F', 'PHI', 'SIGF'], dtype='object')

### 2. Compute a _k_-weighted difference map

Now, we'll compute a _k_-weighted difference map, varying the _k_-parameter to maximize the difference map negentropy.

In [25]:
k_weighted_diffmap, kparameter_metadata = max_negentropy_kweighted_difference_map(
    derivative_map, native_map
)
kewighted_negentropy = map_negentropy(k_weighted_diffmap)

In [26]:
# print(f"optimal k-parameter: {kparameter_metadata.optimal_parameter_value}")
print(f"optimal k-parameter: {kparameter_metadata.optimal_parameter_value}")
print(f"negentropy: {kewighted_negentropy:.5f}")

optimal k-parameter: 0.1
negentropy: -0.00007


### 3. Low pass Gaussian filtering

In [27]:
gaussian_denosied_map, gaussian_metadata = low_pass_denoise_difference_map(k_weighted_diffmap, full_output=True)

Difference map cell paratemers are: 
(51.99, 62.91, 72.03, 90.0, 90.0, 90.0)
Voxel size
[17.33 20.97 24.01]
BRACKET_FOR_GOLDEN_OPTIMIZATION =  (0.0, 5.0)
After normalisaiton NORM_BRACKET_FOR_GOLDEN_OPTIMIZATION =  [0.         0.24073182]
2025-05-02 16:15:50 [info     ] Optimizing Gaussian smoothing sigma using golden-section search.
2025-05-02 16:15:59 [info     ] Applying Gaussian smoothing with normalized sigma normalised_sigma=np.float64(0.18280214881498844) original_sigma=np.float64(0.18280214881498844) voxel_size=np.float64(20.77)


### 4. TV denoising map

In [28]:
tv_denosied_map, tv_metadata = tv_denoise_difference_map(k_weighted_diffmap, full_output=True)

We can inspect the run `metadata` to observe the optimization in progress -- the Golden section method should nicely sample the region around the maximum more densely.

### 5. Compare the methods

In [29]:
tv_metadata.negentropy_gain, gaussian_metadata.negentropy_gain

(0.06575155258178711, 2.384185791015625e-07)

In [30]:
def plot_negentropy_gain_both(tv_metadata, gaussian_metadata, threshold=1e-5):
    if tv_metadata.negentropy_gain < threshold:
        print("Negentropy gain for TV denoising is almost 0! Denoising likely does not improve your map.")
        print(f"Gain: {tv_metadata.negentropy_gain}")
        sys.exit(1)
        
    if gaussian_metadata.negentropy_gain < threshold:
        print("Negentropy gain for Gaussian filtering is almost 0! Filtering likely does not improve your map.")
        print(f"Gain: {gaussian_metadata.negentropy_gain}")
        sys.exit(1)
        
    methods = ["TV", "Gaussian"]
    gains = [tv_metadata.negentropy_gain, gaussian_metadata.negentropy_gain]
    gain_ratios = [tv_metadata.negentropy_gain_ratio, gaussian_metadata.negentropy_gain_ratio]

    fig, axs = plt.subplots(1, 2, figsize=(10, 4))

    # Absolute gain
    axs[0].bar(methods, gains, color=["blue", "red"])
    axs[0].set_ylabel("Negentropy Gain")
    axs[0].set_title("Absolute Improvement")

    # Relative gain
    axs[1].bar(methods, gain_ratios, color=["blue", "red"])
    axs[1].set_ylabel("Negentropy Gain Ratio")
    axs[1].set_title("Relative Improvement")

    for ax in axs:
        ax.set_ylim(0, max(max(gains), max(gain_ratios)) * 1.2)

    plt.tight_layout()
    plt.show()


In [31]:
tv_metadata.negentropy_gain, gaussian_metadata.negentropy_gain

(0.06575155258178711, 2.384185791015625e-07)

In [32]:
plot_negentropy_gain_both(tv_metadata, gaussian_metadata)


Negentropy gain for Gaussian filtering is almost 0! Filtering likely does not improve your map.
Gain: 2.384185791015625e-07


SystemExit: 1

/Users/vapostolop/opt/anaconda3/envs/meteor/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3557: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 4. Write out the results

Awesome! Now we have a fancy new diffmap, which we probably want to save so we can do something with it. Like fire up Coot and check it out. If you are curious, the PDB ID corresponding to this dataset is `8a6g`, and you can find a copy of the structure as a PDB file in this repository:

> `../test/data/8a6g.pdb`

In [ ]:
tv_denosied_map.write_mtz("my_denoised_tv_diffmap.mtz")

In [ ]:
gaussian_denosied_map.write_mtz("my_denoised_gaussian_diffmap.mtz")

In [35]:
vanilla_isomorphous_diffmap.write_mtz("my_denoised_vanilla_diffmap.mtz")